In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
from numpy import asarray

emotion_to_index = {'Angry':0, 'Disgust':1, 'Fear':2, 'Happy':3, 'Neutral':6, 'Sad':4, 'Surprise':5}

def resize_to_224(x):
    train = []
    mat = np.reshape(x[:,:,0], (256, 256))
    src = cv2.resize(mat, dsize=(200,200))
    src = src.astype(np.float16)
    train.append(src)
    mat = np.reshape(x[:,:,1], (256, 256))
    src = cv2.resize(mat, dsize=(200,200))
    src = src.astype(np.float16)
    train.append(src)
    mat = np.reshape(x[:,:,2], (256, 256))
    src = cv2.resize(mat, dsize=(200,200))
    src = src.astype(np.float16)
    train.append(src)
    return train


def getData2(balance_ones=True):

    src = "/content/drive/MyDrive/DATASET/online split"
    Yte = []
    Xte = []
    Xtr = []
    Ytr = []
    Xval = []
    Yval = []

    emotions  =  os.listdir(os.path.join(src,"train"))
    for emotion in emotions:
      print(emotion)
      dir = os.listdir(os.path.join(src,"train",emotion))
      for image in dir:
        img = Image.open("/content/drive/MyDrive/DATASET/online split/train"+"/"+emotion+"/"+image)
        data = asarray(img)
        data = data/255.0
        try:
          train = resize_to_224(data)
        except:
          continue
        Xtr.append(train)
        Ytr.append(emotion_to_index[emotion])
    
    # src = "/content/drive/MyDrive/DATASET/human_split"
    emotions  =  os.listdir(os.path.join(src,"test"))
    for emotion in emotions:
      print(emotion)
      dir = os.listdir(os.path.join(src,"test",emotion))
      for image in dir:
        img = Image.open("/content/drive/MyDrive/DATASET/online split/test"+"/"+emotion+"/"+image)
        data = asarray(img)
        data = data/255.0
        try:
          train = resize_to_224(data)
        except:
          continue
        Xte.append(train)
        Yte.append(emotion_to_index[emotion])

    emotions  =  os.listdir(os.path.join(src,"val"))
    for emotion in emotions:
      print(emotion)
      dir = os.listdir(os.path.join(src,"val",emotion))
      for image in dir:
        img = Image.open("/content/drive/MyDrive/DATASET/online split/val"+"/"+emotion+"/"+image)
        data = asarray(img)
        data = data/255.0
        try:
          train = resize_to_224(data)
        except:
          continue
        Xval.append(train)
        Yval.append(emotion_to_index[emotion])



    Xte, Yte = np.array(Xte, dtype=np.float16), np.array(Yte)
    Xtr, Ytr = np.array(Xtr, dtype=np.float16), np.array(Ytr)
    Xval, Yval = np.array(Xval, dtype=np.float16), np.array(Yval)

    return Xtr, Ytr, Xte, Yte,Xval, Yval

def Reverse(data):
    data = data * -1
    data = data + 1
    return data

In [ ]:
import tensorflow as tf

In [ ]:

from keras.layers import Input, Dense, MaxPooling2D, Flatten, Activation, Embedding, Lambda
from keras.layers import Conv2D
from keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.layers import Layer
from keras import backend as K
from keras import backend
from keras import losses
import numpy as np
import itertools

backend.set_image_data_format('channels_first')

label_size = 7

class IslandLossLayer(Layer):
    def __init__(self, alpha=0.5,lambda_2=0.5, **kwargs):
        super().__init__(**kwargs)
        self.first = True
        self.alpha = alpha
        self.lambda_2 = lambda_2

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers', shape=(7,1024), initializer='uniform', trainable=False)
        super().build(input_shape)

    def L2Norm(ts):
        return K.sqrt(K.sum(ts ** 2, keepdims=True))

    def pairwise_distance(self, pair):
        fea_k = tf.reshape(self.centers[pair[0]], (1,1024))
        fea_j = tf.reshape(self.centers[pair[1]], (1,1024))

        NormK = tf.norm(fea_k, ord=2, axis=1, keepdims=False)
        NormJ = tf.norm(fea_j, ord=2, axis=1, keepdims=False)
        # 1x1
        return K.dot(fea_k, K.transpose(fea_j)) / (NormK * NormJ) + 1


    def pairwise_diff(self, pair):
        Ck = tf.reshape(self.centers[pair[0]], (1,1024))
        Cj = tf.reshape(self.centers[pair[1]], (1,1024))

        NormK = tf.norm(Ck, ord=2, axis=1, keepdims=False)
        NormJ = tf.norm(Cj, ord=2, axis=1, keepdims=False)

        A = Ck / (NormK * NormJ)
        B = (K.dot(Ck, K.transpose(Cj)) / (NormK * (NormJ ** 3))) * Cj

        # 1 x 1024
        T = A - B
        return T

    # x[0] is N*1024 , X[1] is N*7 one hot,
    def call(self, x, mask=None):

        index = np.arange(label_size)
        combination = itertools.permutations(index, 2)

        Centers = K.dot(x[1], self.centers)
        delta_centers = K.dot(K.transpose(x[1]), Centers - x[0])
        center_counts = K.sum(K.transpose(x[1]), axis=1, keepdims=True) + 1
        delta_centers /= center_counts

        # # 7 x N
        # mask_labels = K.transpose(Reverse(x[1]))
        # labels_norm = tf.norm(x[1], ord=2, axis=1, keep_dims=True)
        # A = x[1] / (labels_norm * labels_norm) # Nx7
        # B = K.dot(K.dot(x[1], K.transpose(x[1])) / (labels_norm * (labels_norm ** 3)), x[1]) # K.dot( NxN, Nx7 ) = Nx7
        # add_centers_distance = self.lambda_2 / (label_size-1) * K.dot(K.dot(mask_labels, A - B), self.centers) # 7x1024
        # new_centers = self.centers - self.alpha * delta_centers + add_centers_distance


        # j = 0,1,2,3,4,...label type
        T = []
        for j in range(x[1].shape[1]):
            Sum = tf.zeros(shape=(1, 1024))
            for pair in combination:
                if pair[0] == j:
                    # we need this pair
                    Sum += self.pairwise_diff(pair)
            T.append(Sum)

        add_centers_distance = tf.reshape(tf.stack(T,1), (-1,1024))
        add_centers_distance = (self.lambda_2 / (label_size-1)) * add_centers_distance
        new_centers = self.centers - self.alpha * delta_centers + add_centers_distance
        self.add_update((self.centers, new_centers), x)


        # Center Loss calculate
        self.result = x[0] - K.dot(x[1], self.centers)
        self.result = 0.5 * K.sum(self.result**2, axis=1, keepdims=True)

        # Island Loss calculate
        pairDistance = 0
        for pair in combination:
            pairDistance += self.pairwise_distance(pair)
        pairDistance *= self.lambda_2
        self.result += pairDistance


        # N x 1
        return self.result

    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)


def baseModel(alpha, img, labels):


    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_1')(img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)

    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5')(x)


    model = Model(inputs=img, outputs=x, name='vggface_vgg16')  # load weights
    weights_path = '/content/drive/MyDrive/DATASET/models/rcmalli_vggface_tf_notop_vgg16.h5'
    model.load_weights(weights_path)

    for layer in model.layers[:-4]:
        layer.trainable = False

    base_model_output = model.output
    x = Flatten(name='flatten_final_model')(base_model_output)
    x = Dense(1024, name='fc_final_1')(x)
    x = Activation('relu', name='ss')(x)
    x = Dense(1024, name='fc_final_2')(x)
    x = Activation('relu', name='side_out')(x)

    main = Dense(7, activation='softmax', name='main_out')(x)
    side = IslandLossLayer(alpha=alpha, lambda_2=0.003, name='Ialnslosslayer')([x, labels])

    return main, side





def generateModel2(initial_learning_rate = 1e-3, lambda_c = 0.003, alpha = 0.5):
    main_input = Input(shape=(3, 200,200)) 
    aux_input = Input(shape=(7,))

    Final_output, Side_output = baseModel(alpha, main_input, aux_input)
    model = Model(inputs=[main_input, aux_input], outputs=[Final_output, Side_output])
    model.compile(optimizer=SGD(initial_learning_rate, momentum=0.9),
                  loss=[losses.categorical_crossentropy, lambda y_true, y_pred: y_pred],
                  loss_weights=[1, lambda_c],
                  metrics=['accuracy'])

    return model

In [ ]:
import time

import numpy as np
# ================  Load Data ===================
from keras.utils import np_utils


ontime = time.time()


X_train, Y_train, X_test, Y_test, X_valid, Y_valid = getData2()
print(X_train.shape)
print(Y_train.shape)


y_train_onehot_labels = np_utils.to_categorical(Y_train, 7)
y_test_onehot_lablels = np_utils.to_categorical(Y_test, 7)
y_validation_onehot_labels = np_utils.to_categorical(Y_valid, 7)


# side loss 的设置为0，是因为我们可以在call的时候会根据分类重新计算
# N x 1 为 side loss的维度

y_train_origin_centers = np.zeros((len(X_train), 1))
y_test_origin_centers = np.zeros((len(X_test), 1))
y_valid_origin_centers = np.zeros((len(X_valid), 1))



outup = time.time()
print('Consumption time', outup - ontime)

# ================ Construct Model ===================

# model_centerloss = generateModel1()
model_centerloss = generateModel2(lambda_c=0.2)



# # ================  Model Train & Predict ===================

batch_size = 64
epochs = 75


# 0.65
model_centerloss.fit([X_train, y_train_onehot_labels], [y_train_onehot_labels, y_train_origin_centers], batch_size=batch_size,
                     epochs = epochs, verbose=1,
                     validation_data=([X_valid, y_validation_onehot_labels], [y_validation_onehot_labels, y_valid_origin_centers])
                     )
Loss = model_centerloss.evaluate([X_test, y_test_onehot_lablels], [y_test_onehot_lablels, y_test_origin_centers], verbose=1)
print('Loss: ', Loss)

Fear
Neutral
Happy
Angry
Disgust
Sad
Surprise
Fear
Neutral
Happy
Angry
Disgust
Sad
Surprise
Disgust
Fear
Happy
Neutral
Sad
Surprise
Angry
(1958, 3, 200, 200)
(1958,)
Consumption time 15.498628377914429


Epoch 1/75


31/31 [==============================] - ETA: 0s - loss: 2.1246 - main_out_loss: 1.9313 - Ialnslosslayer_loss: 0.9664 - main_out_accuracy: 0.2273 - Ialnslosslayer_accuracy: 0.0199

31/31 [==============================] - 6s 174ms/step - loss: 2.1246 - main_out_loss: 1.9313 - Ialnslosslayer_loss: 0.9664 - main_out_accuracy: 0.2273 - Ialnslosslayer_accuracy: 0.0199 - val_loss: 2.0567 - val_main_out_loss: 1.9253 - val_Ialnslosslayer_loss: 0.6569 - val_main_out_accuracy: 0.2460 - val_Ialnslosslayer_accuracy: 0.0444
Epoch 2/75
31/31 [==============================] - 5s 156ms/step - loss: 2.0379 - main_out_loss: 1.9182 - Ialnslosslayer_loss: 0.5985 - main_out_accuracy: 0.2380 - Ialnslosslayer_accuracy: 0.1318 - val_loss: 2.0301 - val_main_out_loss: 1.9169 - val_Ialnslosslayer_loss: 0.5659 - val_main_out_accuracy: 0.2097 - val_Ialnslosslayer_accuracy: 0.1573
Epoch 3/75
31/31 [==============================] - 5s 156ms/step - loss: 2.0188 - main_out_loss: 1.9086 - Ialnslosslayer_loss: 0.5512 - main_out_accuracy: 0.2186 - Ialnslosslayer_accuracy: 0.2385 - val_loss: 2.0179 - val_main_out_loss: 1.9093 - val_Ialnslosslayer_loss: 0.5429 - val_main_out_accuracy: 0.2137 - val